### Imports of the Methods

I feel like I may have imported too much

In [1]:
import pandas as pd
import numpy as np
import astropy.units as u
import astropy
from astropy.time import Time
from astropy.coordinates import SkyCoord
import pytz
from astroplan import Observer, FixedTarget
import warnings
warnings.filterwarnings('ignore', category=Warning)
%matplotlib inline
import matplotlib.pyplot as plt
from astroplan import time_grid_from_range
from astroplan.plots import plot_sky, plot_airmass
from astroplan import AltitudeConstraint, AirmassConstraint, AtNightConstraint
from astroplan import observability_table
from astroplan import MoonSeparationConstraint
from operator import itemgetter

### Reading
*In the Data*

In [2]:
data = pd.read_csv(r'browse_results.csv')

Has the data imported correctly?

In [3]:
data[:5]

,name,ra,dec,bjmag,redshift,field_info
0,LBQS 0003+0146,1.4480,2.0504,16.55,0.234,F823
1,LBQS 0003+0011,1.4883,0.4771,18.10,0.259,F823
2,LBQS 0004+0036,1.7916,0.8912,17.79,0.317,F823
3,LBQS 0004+0224,1.8626,2.6866,17.33,0.300,F823
4,LBQS 0006+0015,2.1447,0.5320,17.41,0.263,F823


Send it to the Matrix

In [4]:
data_values = data.as_matrix()
data_values

array([['LBQS 0003+0146', 1.4480000000000002, 2.0504, 16.55,
        0.23399999999999999, 'F823'],
       ['LBQS 0003+0011', 1.4883, 0.4771, 18.1, 0.259, 'F823'],
       ['LBQS 0004+0036', 1.7916, 0.8912, 17.79, 0.317, 'F823'],
       ['LBQS 0004+0224', 1.8626, 2.6866, 17.33, 0.3, 'F823'],
       ['LBQS 0006+0015', 2.1447, 0.532, 17.41, 0.263, 'F823'],
       ['LBQS 0008-0001', 2.7287, 0.2474, 18.41, 0.243, 'F823'],
       ['LBQS 0014+0122', 4.1781, 1.6575, 18.74, 0.29100000000000004,
        'F824'],
       ['LBQS 0015+0119', 4.441, 1.6056, 18.27, 0.23600000000000002,
        'F824'],
       ['LBQS 0016+0105', 4.7095, 1.364, 18.69, 0.319, 'F824'],
       ['LBQS 0017+0212', 5.1383, 2.4811, 18.47, 0.256, 'F824'],
       ['LBQS 0019+0022A', 5.421, 0.6448, 18.64, 0.314, 'F824'],
       ['LBQS 0023+0058', 6.5868, 1.2548, 18.7, 0.27399999999999997,
        'F824'],
       ['LBQS 0023+0228', 6.5914, 2.7449, 18.43, 0.23600000000000002,
        'F824'],
       ['LBQS 0025+0009', 6.9685, 0.4376

In [5]:
targets = [FixedTarget(coord=SkyCoord(ra = RA*u.hourangle, dec = DEC*u.deg), name=Name)
           for Name, RA, DEC, bjmag, redshift, field_info in data_values]

In [6]:
targets

[<FixedTarget "LBQS 0003+0146" at SkyCoord (ICRS): (ra, dec) in deg (21.72, 2.0504)>,
 <FixedTarget "LBQS 0003+0011" at SkyCoord (ICRS): (ra, dec) in deg (22.3245, 0.4771)>,
 <FixedTarget "LBQS 0004+0036" at SkyCoord (ICRS): (ra, dec) in deg (26.874, 0.8912)>,
 <FixedTarget "LBQS 0004+0224" at SkyCoord (ICRS): (ra, dec) in deg (27.939, 2.6866)>,
 <FixedTarget "LBQS 0006+0015" at SkyCoord (ICRS): (ra, dec) in deg (32.1705, 0.532)>,
 <FixedTarget "LBQS 0008-0001" at SkyCoord (ICRS): (ra, dec) in deg (40.9305, 0.2474)>,
 <FixedTarget "LBQS 0014+0122" at SkyCoord (ICRS): (ra, dec) in deg (62.6715, 1.6575)>,
 <FixedTarget "LBQS 0015+0119" at SkyCoord (ICRS): (ra, dec) in deg (66.615, 1.6056)>,
 <FixedTarget "LBQS 0016+0105" at SkyCoord (ICRS): (ra, dec) in deg (70.6425, 1.364)>,
 <FixedTarget "LBQS 0017+0212" at SkyCoord (ICRS): (ra, dec) in deg (77.0745, 2.4811)>,
 <FixedTarget "LBQS 0019+0022A" at SkyCoord (ICRS): (ra, dec) in deg (81.315, 0.6448)>,
 <FixedTarget "LBQS 0023+0058" at SkyCo

#### Hooray
I now have a list of fixed targets to look at.
Set the observer location

In [7]:
aao = Observer.at_site('AAO')

In [8]:
aao

<Observer: name='AAO',
    location (lon, lat, el)=(149.06608611111113 deg, -31.277038888888896 deg, 1164.0000000016162 m),
    timezone=<UTC>>

**It Checks Out** According to Wikipedia!

In [9]:
aao.timezone = pytz.timezone('Australia/Sydney')

In [10]:
aao

<Observer: name='AAO',
    location (lon, lat, el)=(149.06608611111113 deg, -31.277038888888896 deg, 1164.0000000016162 m),
    timezone=<DstTzInfo 'Australia/Sydney' LMT+10:05:00 STD>>

### Timezone
We're in the right timezone!
Window of observation

In [11]:
observe_start = Time("2018-10-15 12:00:00")
observe_end = Time("2018-11-01 12:00:00")

In [12]:
observing_range = [observe_start, observe_end]
constraints = [AirmassConstraint(2), AtNightConstraint(), MoonSeparationConstraint(25*u.deg)]

In [13]:
observing_table = observability_table(constraints, aao, targets, time_range=observing_range)

observing_table

target name,ever observable,always observable,fraction of time observable
str15,bool,bool,float64
LBQS 0003+0146,True,False,0.22426470588235295
LBQS 0003+0011,True,False,0.23039215686274508
LBQS 0004+0036,True,False,0.22916666666666666
LBQS 0004+0224,True,False,0.22549019607843138
LBQS 0006+0015,True,False,0.24754901960784315
LBQS 0008-0001,True,False,0.2340686274509804
LBQS 0014+0122,True,False,0.22549019607843138
LBQS 0015+0119,True,False,0.2267156862745098
LBQS 0016+0105,True,False,0.20710784313725492


In [14]:
mask1 = observing_table['fraction of time observable'] > .225
coordlist = []
#table_observe_sort = observing_table.sort_valuessorted(observing_table, key=itemgetter(3), reverse=True)
observable_table = observing_table[mask1]
print(observable_table)

 target name   ever observable always observable fraction of time observable
-------------- --------------- ----------------- ---------------------------
LBQS 0003+0011            True             False         0.23039215686274508
LBQS 0004+0036            True             False         0.22916666666666666
LBQS 0004+0224            True             False         0.22549019607843138
LBQS 0006+0015            True             False         0.24754901960784315
LBQS 0008-0001            True             False          0.2340686274509804
LBQS 0014+0122            True             False         0.22549019607843138
LBQS 0015+0119            True             False          0.2267156862745098
LBQS 1244-0126            True             False          0.2267156862745098
LBQS 1431+0142            True             False          0.2267156862745098
LBQS 1433-0016            True             False         0.25245098039215685


In [15]:
for i in observable_table['target name']:
    coord = SkyCoord.from_name(i)
    coordlist.append(coord)
    print(coord)

<SkyCoord (ICRS): (ra, dec) in deg
    (1.488523, 0.477165)>
<SkyCoord (ICRS): (ra, dec) in deg
    (1.791724, 0.89142)>
<SkyCoord (ICRS): (ra, dec) in deg
    (1.86256, 2.686657)>
<SkyCoord (ICRS): (ra, dec) in deg
    (2.144651, 0.532275)>
<SkyCoord (ICRS): (ra, dec) in deg
    (2.728555, 0.247597)>
<SkyCoord (ICRS): (ra, dec) in deg
    (4.17797, 1.657545)>
<SkyCoord (ICRS): (ra, dec) in deg
    (4.440985, 1.605679)>
<SkyCoord (ICRS): (ra, dec) in deg
    (191.879038, -1.707726)>
<SkyCoord (ICRS): (ra, dec) in deg
    (218.450896, 1.493778)>
<SkyCoord (ICRS): (ra, dec) in deg
    (219.103393, -0.484825)>


In [28]:
seeable_targets= [
    targets[1],
    targets[2],
    targets[3],
    targets[4],
    targets[5],
    targets[6],
    targets[7],
    targets[58],
    targets[71],
    targets[72]
]
seeable_targets

[<FixedTarget "LBQS 0003+0011" at SkyCoord (ICRS): (ra, dec) in deg (22.3245, 0.4771)>,
 <FixedTarget "LBQS 0004+0036" at SkyCoord (ICRS): (ra, dec) in deg (26.874, 0.8912)>,
 <FixedTarget "LBQS 0004+0224" at SkyCoord (ICRS): (ra, dec) in deg (27.939, 2.6866)>,
 <FixedTarget "LBQS 0006+0015" at SkyCoord (ICRS): (ra, dec) in deg (32.1705, 0.532)>,
 <FixedTarget "LBQS 0008-0001" at SkyCoord (ICRS): (ra, dec) in deg (40.9305, 0.2474)>,
 <FixedTarget "LBQS 0014+0122" at SkyCoord (ICRS): (ra, dec) in deg (62.6715, 1.6575)>,
 <FixedTarget "LBQS 0015+0119" at SkyCoord (ICRS): (ra, dec) in deg (66.615, 1.6056)>,
 <FixedTarget "LBQS 1244-0126" at SkyCoord (ICRS): (ra, dec) in deg (358.185, -1.7078)>,
 <FixedTarget "LBQS 1431+0142" at SkyCoord (ICRS): (ra, dec) in deg (36.7605, 1.4937)>,
 <FixedTarget "LBQS 1433-0016" at SkyCoord (ICRS): (ra, dec) in deg (46.551, -0.4847)>]

In [17]:
observable_table['SkyCoord'] = coordlist
observable_table

target name,ever observable,always observable,fraction of time observable,SkyCoord
str15,bool,bool,float64,object
LBQS 0003+0011,True,False,0.23039215686274508,"<SkyCoord (ICRS): (ra, dec) in deg (1.488523, 0.477165)>"
LBQS 0004+0036,True,False,0.22916666666666666,"<SkyCoord (ICRS): (ra, dec) in deg (1.791724, 0.89142)>"
LBQS 0004+0224,True,False,0.22549019607843138,"<SkyCoord (ICRS): (ra, dec) in deg (1.86256, 2.686657)>"
LBQS 0006+0015,True,False,0.24754901960784315,"<SkyCoord (ICRS): (ra, dec) in deg (2.144651, 0.532275)>"
LBQS 0008-0001,True,False,0.2340686274509804,"<SkyCoord (ICRS): (ra, dec) in deg (2.728555, 0.247597)>"
LBQS 0014+0122,True,False,0.22549019607843138,"<SkyCoord (ICRS): (ra, dec) in deg (4.17797, 1.657545)>"
LBQS 0015+0119,True,False,0.2267156862745098,"<SkyCoord (ICRS): (ra, dec) in deg (4.440985, 1.605679)>"
LBQS 1244-0126,True,False,0.2267156862745098,"<SkyCoord (ICRS): (ra, dec) in deg (191.879038, -1.707726)>"
LBQS 1431+0142,True,False,0.2267156862745098,"<SkyCoord (ICRS): (ra, dec) in deg (218.450896, 1.493778)>"


### Explaination
Skycoord could not actually look them all up by name so I looked for the names in the big list and counted. All the coords in the list above are wrong. List list below is right.

In [29]:
seeable_targets

[<FixedTarget "LBQS 0003+0011" at SkyCoord (ICRS): (ra, dec) in deg (22.3245, 0.4771)>,
 <FixedTarget "LBQS 0004+0036" at SkyCoord (ICRS): (ra, dec) in deg (26.874, 0.8912)>,
 <FixedTarget "LBQS 0004+0224" at SkyCoord (ICRS): (ra, dec) in deg (27.939, 2.6866)>,
 <FixedTarget "LBQS 0006+0015" at SkyCoord (ICRS): (ra, dec) in deg (32.1705, 0.532)>,
 <FixedTarget "LBQS 0008-0001" at SkyCoord (ICRS): (ra, dec) in deg (40.9305, 0.2474)>,
 <FixedTarget "LBQS 0014+0122" at SkyCoord (ICRS): (ra, dec) in deg (62.6715, 1.6575)>,
 <FixedTarget "LBQS 0015+0119" at SkyCoord (ICRS): (ra, dec) in deg (66.615, 1.6056)>,
 <FixedTarget "LBQS 1244-0126" at SkyCoord (ICRS): (ra, dec) in deg (358.185, -1.7078)>,
 <FixedTarget "LBQS 1431+0142" at SkyCoord (ICRS): (ra, dec) in deg (36.7605, 1.4937)>,
 <FixedTarget "LBQS 1433-0016" at SkyCoord (ICRS): (ra, dec) in deg (46.551, -0.4847)>]